# ML Assigment 2
Spambase, Anna Bergknut, DVAMI21

In [1]:
import matplotlib.pyplot as mpl
import pandas as pd
import numpy as np
import scipy as scp
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split

from scipy.stats import friedmanchisquare, rankdata, norm, chi2, ranksums, kruskal

In [2]:
file_path = "spambase.data"
columns = ['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 'word_freq_our', 'word_freq_over', 'word_freq_remove', 'word_freq_internet', 'word_freq_order', 'word_freq_mail', 'word_freq_receive', 'word_freq_will', 'word_freq_people', 'word_freq_report', 'word_freq_addresses', 'word_freq_free', 'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit', 'word_freq_your', 'word_freq_font', 'word_freq_000', 'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 'word_freq_george', 'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet', 'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85', 'word_freq_technology', 'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct', 'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project', 'word_freq_re', 'word_freq_edu', 'word_freq_table', 'word_freq_conference', 'charfreq;', 'charfreq(', 'charfreq[', 'charfreq!', 'charfreq$', 'charfreq#', 'capital_run_length_average', 'capital_run_length_longest', 'capital_run_length_total', 'spam/nonspam']


In [3]:
# Load Data
data = pd.read_csv(file_path, header=None, names=columns, sep=",")

# check if null
data.isnull().values.any()

False

In [4]:
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Define 10-fold stratified cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [6]:
# Decision Tree
decision_tree_model = DecisionTreeClassifier(random_state=42)

%timeit decision_tree_model.fit(X_train, y_train)
dt_predictions = decision_tree_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)
dt_f_measure = f1_score(y_test, dt_predictions, average='binary')
print(f'Decision Tree Accuracy: {dt_accuracy}')
print(f'Decision Tree F-Measure: {dt_f_measure}')

75.6 ms ± 982 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Decision Tree Accuracy: 0.9185667752442996
Decision Tree F-Measure: 0.9019607843137256


In [7]:
# k-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier()

%timeit knn_model.fit(X_train, y_train)
knn_predictions = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_f_measure = f1_score(y_test, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy}')
print(f'Random Forest F-Measure: {knn_f_measure}')

2.78 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
KNN Accuracy: 0.7904451682953312
Random Forest F-Measure: 0.7423230974632844


In [8]:
# Random Forest
random_forest_model = RandomForestClassifier(random_state=42)

%timeit random_forest_model.fit(X_train, y_train)
rf_predictions = random_forest_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_f_measure = f1_score(y_test, rf_predictions, average='binary')
print(f'Random Forest Accuracy: {rf_accuracy}')
print(f'Random Forest F-Measure: {rf_f_measure}')

836 ms ± 23.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Random Forest Accuracy: 0.9554831704668838
Random Forest F-Measure: 0.9458388375165125


In [12]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(index=range(1, 11))

# Loop through each fold
for fold in range(1, 11):
    # Decision Tree
    dt_scores = cross_val_score(decision_tree_model, X, y, cv=cv, scoring='accuracy')
    results_df.at[fold, 'Decision Tree'] = dt_scores.mean()
    
    # KNN
    knn_scores = cross_val_score(knn_model, X, y, cv=cv, scoring='accuracy')
    results_df.at[fold, 'k-Nearest Neighbors'] = knn_scores.mean()

    # Random Forest
    rf_scores = cross_val_score(random_forest_model, X, y, cv=cv, scoring='accuracy')
    results_df.at[fold, 'Random Forest'] = rf_scores.mean()

# Calculate average and standard deviation
results_df.at['avg', :] = results_df.mean(axis=0)
results_df.at['stdev', :] = results_df.std(axis=0)

# Display the results
print(results_df)

       Decision Tree  k-Nearest Neighbors  Random Forest
1           0.910679         8.096081e-01   9.547930e-01
2           0.910679         8.096081e-01   9.547930e-01
3           0.910679         8.096081e-01   9.547930e-01
4           0.910679         8.096081e-01   9.547930e-01
5           0.910679         8.096081e-01   9.547930e-01
6           0.910679         8.096081e-01   9.547930e-01
7           0.910679         8.096081e-01   9.547930e-01
8           0.910679         8.096081e-01   9.547930e-01
9           0.910679         8.096081e-01   9.547930e-01
10          0.910679         8.096081e-01   9.547930e-01
avg         0.910679         8.096081e-01   9.547930e-01
stdev       0.000000         1.164412e-16   2.248030e-16


In [10]:
# Conduct the Friedman test
_, p_value = friedmanchisquare(
    results_df['Decision Tree'], 
    results_df['k-Nearest Neighbors'],  # Corrected column name
    results_df['Random Forest']
)
print(f'Friedman test p-value: {p_value}')

# If p-value is significant, proceed with Nemenyi test
if p_value < 0.05:
    # Function to calculate the critical difference using Nemenyi test
    def nemenyi_critical_difference(rank_avg, num_algorithms, num_samples):
        q_val = chi2.ppf(1 - 0.05 / (2 * num_algorithms * (num_algorithms - 1)) ** 0.5, df=num_algorithms - 1)
        return q_val * (num_algorithms * (num_algorithms + 1) / (6 * num_samples)) ** 0.5

    # Rank the algorithms
    ranks = rankdata(results_df.iloc[:-2, :].values, axis=1).mean(axis=0)

    # Calculate critical difference
    num_algorithms = results_df.shape[1]
    num_samples = results_df.shape[0] - 2
    cd = nemenyi_critical_difference(ranks, num_algorithms, num_samples)
    print(f'Critical Difference: {cd}')

Friedman test p-value: 1.5366306526875732e-05
Critical Difference: 3.7907484397561317


In [11]:
# Calculate average ranks
average_ranks = rankdata(results_df.iloc[:-2, :].values, axis=1).mean(axis=0)

# Calculate rank sums for each algorithm
rank_sums = [np.sum(ranks) for ranks in results_df.iloc[:-2, :].values]

# Calculate H-statistic for the Kruskal-Wallis test
H_statistic, _ = kruskal(*results_df.iloc[:-2, :].values)

# Calculate critical value for the Kruskal-Wallis test
k = results_df.shape[1]  # Number of algorithms
n = results_df.shape[0] - 2  # Number of samples (folds)
critical_value = 3.841  # From the chi-squared distribution table for alpha = 0.05 and df = k - 1

# Determine whether the average ranks display significant differences
if H_statistic > critical_value:
    print("The average ranks display significant differences.")

    # Use Nemenyi test to calculate critical difference
    cd_nemenyi = nemenyi_critical_difference(average_ranks, k, n)
    print(f'Critical Difference (Nemenyi): {cd_nemenyi}')

    # Identify which algorithms perform significantly differently from each other
    significant_pairs = []
    for i in range(k - 1):
        for j in range(i + 1, k):
            diff = np.abs(average_ranks[i] - average_ranks[j])
            if diff > cd_nemenyi:
                significant_pairs.append((i, j))

    print("Significantly different algorithm pairs:")
    for pair in significant_pairs:
        algorithm1, algorithm2 = pair
        print(f"{results_df.columns[algorithm1]} and {results_df.columns[algorithm2]}")

else:
    print("The average ranks do not display significant differences.")

The average ranks do not display significant differences.
